In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/home/kevin/Downloads/titanic.csv')


In [4]:
df.loc[df['Sex'] == 'female', 'Survived'].value_counts()

1    233
0     81
Name: Survived, dtype: int64

In [5]:
a = df.groupby(['Survived', 'Sex']).size().reset_index()

In [6]:
a.pivot(index='Sex', columns='Survived', values=0)

Survived,0,1
Sex,,
female,81,233
male,468,109


In [7]:
pd.crosstab(df['Sex'], df['Survived'], normalize='index')

Survived,0,1
Sex,,
female,0.257962,0.742038
male,0.811092,0.188908


In [13]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')